In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
ops = ['STORE', 'PLUS', 'MINUS', 'TIMES', 'DIV', 'INCR', 'EQ?', 'NEQ?', 'BR', 'BZ', 'BNZ', 'DUP', 'POP', 'DISP']

# i = 1; while i != 10 { i++; }
code1 = [ ['NUM',1], ['STORE','i'], ['LDEF','L1'], ['RVAL','i'], ['NUM',10], ['NEQ?'], ['LREF','L2'], ['BZ'], ['INCR','i'], ['LREF','L1'], ['BR'], ['LDEF','L2'], ['DISP2']]

# s = 0; i = 1; while i != 11 { s = s + i; i++; }
code2 = ['s', 0, 'STORE', 'i', 1, 'STORE', 'L1:', 'i', 101, 'NEQ?', 'L2!', 'BZ', 's' , 's', 'i', 'PLUS', 'STORE', 'i', 'INCR', 'L1!', 'BR', 'L2:', 'DISP2']

# i = 5; j = i++; k = ++i; 
code3 = [ ['NUM',5], ['STORE','i'], ['RVAL','i'], ['INCR','i'], ['STORE', 'j'], ['INCR','i'], ['RVAL','i'], ['STORE', 'k'], ['DISP2']]

# i = 5; k = i*i++
code4 = [ ['NUM',5], ['STORE','i'], ['RVAL','i'], ['RVAL','i'] , ['INCR','i'], ['TIMES'], ['STORE', 'j'], ['DISP2']]
code4a = [ ['NUM',5], ['STORE','i'], ['RVAL','i'], ['DUP'] , ['INCR','i'], ['TIMES'], ['STORE', 'j'], ['DISP2']]

In [3]:
class PostfixVM :
    def __init__(self, ops, code) :
        self.stack = None                    # Working stack
        self.env = None                      # Bindings
        self.ops = ops                       # List of op symbols
        self.code = code                     # Code to execute
        self.CONT = -1                       # Continuation counter
    
    def PUSH(self,value) : 
        self.stack.append(value)
    
    def POP(self) :
        return self.stack.pop()
    
    def ensureBound(self,key) :
        value = self.env.get(key,None)
        if (value != None) : return value
        self.env[key] = 0
        return 0
    
    def STORE(self,key,value) :
        self.env[key] = value
        
    def EVAL(self, atom) :
        if atom[0] == 'NUM' : return atom[1] # constants evaluate to themselves
        # Must of type LVAL now: Make sure it's in the environment and return the value
        return self.ensureBound(atom[1])
    
    def display(self) :
        print(self.CONT,self.code[self.CONT])
        print(self.env)
        print(self.stack)
    
    def show(self,rvalue) :
        print(self.EVAL(rvalue))
    
    def run(self) :
        self.CONT = 0;
        self.stack = []
        self.env = {}
        while self.CONT < len(self.code) : # Still inside the code
            op = self.code[self.CONT]
            self.CONT += 1
            self.doOp(op)
            self.display()

    def position(self,lref) :
        return self.code.index(['LDEF',lref[1]])
            
    def doOp(self,op) :
        # print('OP: ',op)
        
        # Stack OPs
        
        if op[0] in ['LVAL', 'NUM', 'LREF'] : self.PUSH(op)
            
        
        if op[0] == 'RVAL' : self.PUSH(['NUM',self.EVAL(op)])
        if op[0] == 'DISP1' : self.show(self.POP())
        if op[0] == 'DISP2' : self.display()
        if op[0] == 'STORE' :
            v = self.EVAL(self.POP())
            self.STORE(op[1],v)
            
        # Arithmetic
                
        if op[0] == 'PLUS' :
            x = self.EVAL(self.POP())
            self.PUSH(['NUM',self.EVAL(self.POP()) + x])
        if op[0] == 'TIMES' :
            x = self.EVAL(self.POP())
            self.PUSH(['NUM',self.EVAL(self.POP()) * x])
        if op[0] == 'MINUS' :
            x = self.EVAL(self.POP())
            self.PUSH(['NUM',self.EVAL(self.POP()) - x])
        if op[0] == 'DIV' :
            x = self.EVAL(self.POP())
            self.PUSH(['NUM',self.EVAL(self.POP()) / x])

        if op[0] == 'INCR' :
            var = op[1]
            self.STORE(var,self.EVAL(['RVAL',var]) + 1)
        if op[0] == 'DECR' :
            var = op[1]
            self.STORE(var,self.EVAL(['RVAL',var]) + 1)

                
        # Comparisons
                
        if op[0] == 'EQ?' :
            v = self.EVAL(self.POP()) == self.EVAL(self.POP())
            self.PUSH(['NUM',1 if v else 0])
        if op[0] == 'NEQ?' :
            v = self.EVAL(self.POP()) == self.EVAL(self.POP())
            self.PUSH(['NUM',0 if v else 1])
                
        # Branches
                
        if op[0] == 'BR' :
            self.CONT = self.position(self.POP())

        if op[0] == 'BZ' :
            label = self.POP()
            if self.EVAL(self.POP()) == 0 :
                self.CONT = self.position(label)
        if op[0] == 'BNZ' :
            label = self.POP()
            if self.EVAL(self.POP()) != 0 :
                self.CONT = self.position(label)
        return
        

In [4]:
pv = PostfixVM(ops,code4)
print(pv.code)
pv.run()

[['NUM', 5], ['STORE', 'i'], ['RVAL', 'i'], ['RVAL', 'i'], ['INCR', 'i'], ['TIMES'], ['STORE', 'j'], ['DISP2']]
1 ['STORE', 'i']
{}
[['NUM', 5]]
2 ['RVAL', 'i']
{'i': 5}
[]
3 ['RVAL', 'i']
{'i': 5}
[['NUM', 5]]
4 ['INCR', 'i']
{'i': 5}
[['NUM', 5], ['NUM', 5]]
5 ['TIMES']
{'i': 6}
[['NUM', 5], ['NUM', 5]]
6 ['STORE', 'j']
{'i': 6}
[['NUM', 25]]
7 ['DISP2']
{'i': 6, 'j': 25}
[]


IndexError: list index out of range